### Tim Moriarity Capstone

For my capstone I plan on generating projections for fantasy football, specifically quarterbacks.  If there is time I will expand to the other positions, but intitally the focus will be on quarterbacks. I plan to generate my model from historical football data.  I will convert actual game statistics into fantasy points to compare the accuracy of my projections, witht the end result being projections for unplayed weeks.

I will draw my data from https://www.pro-football-reference.com which will give me data for each week of the 3 NFL seasons going back through 2015. This gives me access to all the historical stats from the NFL games in the last 3 years.  I will intially try to use an API to gather the data, but if necessary I can manually acquire the data I need. The site has access to data by position as well as, defensive data that will be included in calculating the projections.

I plan to use the player statistics to calculate their past fantasy scores and compare them to the projections I made for that week. I will have to combine multiple datasets to generate the correct dataframe to create projections.  

I will use the weekly defensive player stats generate defensive variables that impact the projections. I will take into account the past play up to 3 years for players in the league. Rookies and Sophmores will be taken into account using dummy tags.  I plan to use whatever domain knowledge on projection generation I can find, as well as my own for independent variable generation.

In [286]:
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
from datetime import datetime
from statsmodels.tsa.arima_model import ARMA, AR, ARIMA, _arma_predict_out_of_sample, ARMAResults
from statsmodels.tsa.stattools import adfuller

In [287]:
file_lst = os.listdir('./raw_data_nfl_QB/')### create directory folder to loop through

In [288]:
for idx,f in enumerate(file_lst):
    if idx == 0:
        data = pd.read_csv('./raw_data_nfl_QB/{}'.format(f))
    else:
        temp_df = pd.read_csv('./raw_data_nfl_QB/{}'.format(f))
        data = pd.concat([data,temp_df],axis = 0)

In [289]:
data.reset_index(inplace=True)

In [78]:
data.shape

(2406, 24)

In [79]:
data.columns

Index(['index', 'Rk', 'Unnamed: 1', 'Age', 'Date', 'Lg', 'Tm', 'Unnamed: 6',
       'Opp', 'Result', 'G#', 'Week', 'Day', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD',
       'Int', 'Rate', 'Sk', 'Yds.1', 'Y/A', 'AY/A'],
      dtype='object')

convert date to date time object for time series set as index 

drop low completion numbers

aggregate by weeks over years

label qbs by year

check for stationary if not turn into stationary

    stats models package
    
    check lecture

In [290]:
data['Year'] = data['Date'].str.split('-', 1, expand=True)[0]
data['Year'] = data['Year'].astype(str).astype(float)
data['Month'] = data['Date'].str.split('-', 2, expand=True)[1]
data['Month'] = data['Month'].astype(str).astype(float)
data['Name'] = data['Unnamed: 1'].str.split('\\', 1, expand=True)[0]
data['Home_Away'] = data['Unnamed: 6']
data['Home_Away'] = (data['Home_Away']=='@').map({False:'Home', True:'Away'})
data['Home_Team'] = (data['Home_Away']=='Home').map({True:1, False:0})

In [291]:
from statsmodels.tsa.arima_model import ARMAResults

In [292]:
data = data.loc[data['Att']>=5]

In [138]:
dataint = data[[col for col in data.columns if data[col].dtype!=object]].copy()

In [145]:
dataint.columns

Index(['index', 'Rk', 'G#', 'Week', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'Int',
       'Rate', 'Sk', 'Yds.1', 'Y/A', 'AY/A'],
      dtype='object')

In [113]:
time_int = dataint.iloc[:, 3:]

In [114]:
time_int.head()

,Week,Cmp,Att,Cmp%,Yds,TD,Int,Rate,Sk,Yds.1,Y/A,AY/A
0,1,27,42,64.29,462,7,0,141.1,3,17,11.00,14.33
1,1,27,39,69.23,412,3,0,129.4,2,8,10.56,12.10
2,1,18,23,78.26,178,2,0,127.9,4,31,7.74,9.48
3,1,25,33,75.76,320,1,0,115.7,2,20,9.70,10.30
4,1,26,35,74.29,357,2,1,113.6,2,16,10.20,10.06


In [96]:
time_int_diff_df = time_int.diff()
time_int_diff_df.dropna(inplace=True)

In [225]:
X = dataint[['Week', 'Cmp', 'Att', 'Cmp%', 'TD', 'Int','Rate', 'Sk']]
y = dataint['Yds'].values

In [270]:
pd.DatetimeIndex(X['Week'], date=X['Week'], start=1, end=17)

DatetimeIndex(['1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               '1970-01-01 00:00:00.000000001',
               ...
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009',
               '1970-01-01 00:00:00.000000009'],
              dtype=

In [227]:
lr = LinearRegression()
lr.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [228]:
predict1 = lr.predict(X)

In [229]:
residuals = predict1-y.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [272]:
data.set_index('Week', verify_integrity=True)

ValueError: Index has duplicate keys: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [235]:
import numpy as np

In [242]:
y = y.astype(str).astype(float)

In [252]:
ar_yards = ARMA(residuals, (3,1)).fit()
params = ar_yards.params

In [205]:
params

array([-0.00338282, -0.69644521,  0.05138966,  0.02644694,  0.73948423])

In [266]:
pd.Period(data['Week'])

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [263]:
pd.PeriodIndex(data['Week'], start=1, end=17, periods=17)

ValueError: freq not specified and cannot be inferred

In [256]:
X['Week'] = pd.to_datetime(X['Week'], format=X['Week'])

TypeError: startswith first arg must be str or a tuple of str, not Series

In [200]:
res_pred = ar_yards.predict()

In [254]:
params = ar_yards.params
residuals = ar_yards.resid
p = ar_yards.k_ar
q = ar_yards.k_ma
k_exog = ar_yards.k_exog
k_trend = ar_yards.k_trend
steps = 1

oos_predictions = _arma_predict_out_of_sample(params, steps, residuals, 
                                p, q, k_trend, k_exog, 
                                endog=ar_yards, exog=None, start=0)

TypeError: 'ARMAResultsWrapper' object is not subscriptable

In [197]:
ARMAResults(ar_yards, )

AttributeError: 'ARMAResults' object has no attribute 'endog'

In [133]:
results

(-13.612649894444555,
 1.8604700783576195e-25,
 18,
 2387,
 {'1%': -3.4330924994550207,
  '10%': -2.5674149839928577,
  '5%': -2.8627515964817447},
 28730.905637386742)

In [119]:
results_Yds = adfuller(data['Yds'].values)
results_TD = adfuller(data['TD'].values)
results_Int = adfuller(data['Int'].values)

In [132]:
print('Yards Dickey Fuller Test')
print(results_Yds[0], 'T-Statistic')
print(results_Yds[1], 'P_value')
print(results_Yds[4], 'Critical Values')
print('TD Dickey Fuller Test')
print(results_TD[0], 'T-Statistic')
print(results_TD[1], 'P_value')
print(results_TD[4], 'Critical Values')
print('Int Dickey Fuller Test')
print(results_Int[0], 'T-Statistic')
print(results_Int[1], 'P_value')
print(results_Int[4], 'Critical Values')

Yards Dickey Fuller Test
-13.6126498944 T-Statistic
1.86047007836e-25 P_value
{'1%': -3.4330924994550207, '5%': -2.8627515964817447, '10%': -2.5674149839928577} Critical Values
TD Dickey Fuller Test
-14.4415663463 T-Statistic
7.34472764758e-27 P_value
{'1%': -3.4331005743547789, '5%': -2.8627551622102132, '10%': -2.5674168824588657} Critical Values
Int Dickey Fuller Test
-15.3205391572 T-Statistic
4.06989659898e-28 P_value
{'1%': -3.4330994178876191, '5%': -2.862754651535997, '10%': -2.5674166105654788} Critical Values


In [181]:
for i in data.Name:
    i.str.split(\)

SyntaxError: unexpected character after line continuation character (<ipython-input-181-c413435cde65>, line 2)

In [96]:
data.shape

(2270, 25)

In [143]:
data['Rank_QBR'] = data['Rk']
data['Name'] = data['Unnamed: 1']
data['Sack'] = data['Sk']
data['Completions'] = data['Cmp']
data['Attempts'] = data['Att']
data['Passing_Yards'] = data['Yds']
data['Sack_Yards'] = data['Yds.1']
data['QBR'] = data['Rate']
data['Yds_Per_Attempt'] =  data['Y/A']
data['Adj_Yd_Per_Attempt'] = data['AY/A']

In [31]:
drop_cols = ['Rk', 'Unnamed: 1', 'Sk', 'Cmp', 'Yds', 'Lg', 'Unnamed: 6', 'Att', 'Yds', 'Yds.1', 'Rate', 'Y/A', 'AY/A']
data = data.drop(drop_cols, axis=1, inplace=True)

TypeError: 'NoneType' object is not subscriptable

In [66]:
data['QB_ID'] = QB_ID

In [64]:
QB_ID = []
for a, b in zip(data['Year'], data['Name']):
    Qid = '{}_{}'.format(a,b)
    QB_ID.append(Qid)

data['QB_ID'] = QB_ID

In [70]:
data['QB_ID'].head()

0      2013_Peyton Manning
1    2013_Colin Kaepernick
2         2013_Andrew Luck
3      2013_Russell Wilson
4          2013_Drew Brees
Name: QB_ID, dtype: object

In [100]:
Week1_df = data.loc[data['Week']==1]
Week2_df = data.loc[data['Week']==2]
Week3_df = data.loc[data['Week']==3]
Week4_df = data.loc[data['Week']==4]
Week5_df = data.loc[data['Week']==5]
Week6_df = data.loc[data['Week']==6]
Week7_df = data.loc[data['Week']==7]
Week8_df = data.loc[data['Week']==8]
Week9_df = data.loc[data['Week']==9]
Week10_df = data.loc[data['Week']==10]
Week11_df = data.loc[data['Week']==11]
Week12_df = data.loc[data['Week']==12]
Week13_df = data.loc[data['Week']==13]
Week14_df = data.loc[data['Week']==14]
Week15_df = data.loc[data['Week']==15]
Week16_df = data.loc[data['Week']==16]
Week17_df = data.loc[data['Week']==17]

In [101]:
Week11_df.shape

(122, 25)

In [ ]:
_arma_predict_out_of_sample

In [99]:
Week1_df.shape

(128, 25)

In [216]:
df_2016 = data.loc[data['Year']==2016]
df_2015 = data.loc[data['Year']==2015]
df_2014 = data.loc[data['Year']==2014]
df_2013 = data.loc[data['Year']==2013]

In [217]:
df_2013

,index,Rk,Unnamed: 1,Age,Date,Lg,Tm,Unnamed: 6,Opp,Result,...,TD,Int,Rate,Sk,Yds.1,Y/A,AY/A,Year,Month,Name


In [284]:
df_2013.set_index('Week', inplace=True)

KeyError: 'Week'

In [285]:
df_2013

,index,Rk,Unnamed: 1,Age,Date,Lg,Tm,Unnamed: 6,Opp,Result,...,TD,Int,Rate,Sk,Yds.1,Y/A,AY/A,Year,Month,Name
Week,,,,,,,,,,,,,,,,,,,,,
